# Topsis 评价模型

## 模型

形象根据距离概念，给出多指标下的综合评分

要点在于找出理想最优/最劣点，再求各方案与两者的距离

### 正向化

指标 $\rightarrow$ **极大型**指标

指标分为：

- 极大型：越大越好

    $\hat{x} = x$

- 极小型：越小越好

    $\hat{x} = \max - x$

- 中间型：越靠近某值越好

    $\hat{x}_{i} = 1 - \frac{\left|x_{i} - x_{best}\right|}{\max \{ \left|x_{i} - x_{best}\right| \}}$

- 区间型：落在某区间最好

    ![](./topsis_1.png)

### 标准化

消除量纲影响

## 预实现

In [7]:
import numpy as np

# 正向化处理

## 处理极小型
def handle_min_best(x: np.ndarray):
    return x.max(0) - x

## 处理中间型
def handle_middle_best(x: np.ndarray, best):
    return 1 - np.abs(x - best) / np.max(np.abs(x - best))

## 处理区间型
def handle_area_best(x: np.ndarray, l, r):
    M = np.append(l - np.min(x), np.max(x) - r).max()
    return \
        np.where(
            x < l, 
            1 - (l - x) / M,
            np.where(
                x > r, 
                1 - (x - r) / M,
                1      
            ))
        
## 标准化
def standardize(x: np.ndarray):
    tep1 = (x * x).sum(axis=0) # 每个元素平方后按列相加
    tep2 = np.tile(tep1, (x.shape[0], 1))  # 将矩阵tep_x1平铺n行
    return x / ((tep2) ** 0.5)

## 实现

In [8]:
import numpy as np
# import pandas as pd
# import openpyxl as xl

#! 修改数据读入
data = np.array([[1, 2, 3, 4, 5, 9, 11],[1, 2, 100, 4, 2, 9, 11],[1, 44, 3, 4, 56, 9, 11]],dtype=float)
# data = np.array(pd.read_csv("data.csv"))
# data = np.array(xl.open("data.xlsl"))

#! 修改正向化
data[0] = handle_middle_best(data[0], 5)
data[1] = handle_min_best(data[1])
data[2] = handle_area_best(data[2], 3, 7)

print(data)

# 标准化
data = standardize(data)

print("预处理后数据：\n",data)

[[ 0.33333333  0.5         0.66666667  0.83333333  1.          0.33333333
   0.        ]
 [99.         98.          0.         96.         98.         91.
  89.        ]
 [ 0.95918367  0.24489796  1.          1.          0.          0.95918367
   0.91836735]]
预处理后数据：
 [[0.00336683 0.00510196 0.5547002  0.00867976 0.01020355 0.00366278
  0.        ]
 [0.9999474  0.99998386 0.         0.99990808 0.99994794 0.99993775
  0.99994677]
 [0.00968821 0.00249892 0.83205029 0.01041571 0.         0.01053982
  0.01031818]]


In [9]:
n = data.shape[0]
dmax = data.max(0)  # 得到data中每列的最大值
dmin = data.min(0)  # 每列的最小值

tep_a = data - np.tile(dmax, (n, 1))  # 将tep_max向下平铺n行,并与data中的每个对应元素做差
tep_i = data - np.tile(dmin, (n, 1))  # 将tep_max向下平铺n行，并与data中的每个对应元素做差

D_P = ((tep_a ** 2).sum(axis=1)) ** 0.5  # D+与最大值的距离向量
D_N = ((tep_i ** 2).sum(axis=1)) ** 0.5
S = D_N / (D_P + D_N)  # 未归一化的得分

std_S = S / S.sum(axis=0)
# sorted_S = np.sort(std_S, axis=0)

print("TOPSIS综合得分:\n", std_S)  # 打印标准化后的得分

TOPSIS综合得分:
 [0.15564449 0.62924943 0.21510608]
